In [6]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [7]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [8]:
tf.keras.backend.clear_session()

In [9]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [10]:
subject = 'Roraima - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [12]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_RR.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data = data.drop('Roraima - Produção de Cimento (t)', axis=1)
data

,Ano,Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado,Roraima - IDH,Roraima - IDH Longevidade,Roraima - IDH Educacao,Roraima - IDH Renda,Contratação Comercial (Bi R$),...,Taxa Selic (%),Roraima - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Roraima - Consumo de Cimento (t)
0,2008,6.555833e+06,342849.112404,12.235418,5.474033e+06,0.736454,0.752834,0.733615,0.722576,5.483335e+08,...,11.760983,8.015090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.009000
1,2009,6.611657e+06,345335.485288,12.228462,5.482306e+06,0.736503,0.754681,0.734851,0.720114,5.302764e+08,...,9.472585,8.046321,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.064000
2,2010,6.787712e+06,348962.483930,12.215749,5.534487e+06,0.736471,0.756148,0.735694,0.718135,5.046425e+08,...,9.332485,8.009229,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86.408000
3,2011,6.947966e+06,347630.972013,12.206783,5.574553e+06,0.736360,0.757242,0.736139,0.716628,4.854622e+08,...,10.985689,7.996819,22.217634,1001.541199,1506.456126,27.425647,21.330889,71.595088,1.830447,103.009000
4,2012,7.096137e+06,347411.232273,12.186539,5.609510e+06,0.736089,0.757859,0.736249,0.715498,4.867661e+08,...,8.159014,8.651837,21.157249,1001.690342,1515.527356,27.608612,21.117524,70.061650,1.898255,100.066000
5,2013,7.239848e+06,349584.100654,12.182243,5.648192e+06,0.735929,0.758347,0.735436,0.715372,5.225848e+08,...,7.913561,9.551098,18.013769,1001.817868,1528.998642,27.762053,20.876766,68.552790,1.943125,121.914000
6,2014,7.349320e+06,344289.736627,12.154911,5.667961e+06,0.735757,0.758460,0.735472,0.714737,5.676413e+08,...,10.350442,10.328300,14.644773,1001.940536,1542.131372,27.986329,20.647922,66.896135,1.956684,120.417571
7,2015,7.439258e+06,340498.356490,12.130631,5.685185e+06,0.735070,0.758081,0.733678,0.714965,5.179392e+08,...,12.454398,10.723767,19.007683,1001.988306,1541.272845,28.048825,20.665808,66.738045,1.910770,113.282057
8,2016,7.513614e+06,340512.038701,12.120014,5.705074e+06,0.735168,0.766432,0.738349,0.702779,4.524144e+08,...,13.111908,10.764101,24.437747,1002.055065,1514.905960,27.959130,20.665948,67.018448,1.902067,105.481371
9,2017,7.560265e+06,340005.357917,12.115483,5.723617e+06,0.733637,0.765744,0.735380,0.701759,4.213689e+08,...,9.479812,10.533542,26.318142,1002.155784,1495.611624,27.867978,20.547545,66.903661,1.943183,99.723000


In [13]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado,Roraima - IDH,Roraima - IDH Longevidade,Roraima - IDH Educacao,Roraima - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Roraima - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,6.555833e+06,342849.112404,12.235418,5.474033e+06,0.736454,0.752834,0.733615,0.722576,5.483335e+08,2.546644e+09,...,2.868108,11.760983,8.015090,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6.611657e+06,345335.485288,12.228462,5.482306e+06,0.736503,0.754681,0.734851,0.720114,5.302764e+08,2.679945e+09,...,2.305885,9.472585,8.046321,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6.787712e+06,348962.483930,12.215749,5.534487e+06,0.736471,0.756148,0.735694,0.718135,5.046425e+08,2.891930e+09,...,1.958246,9.332485,8.009229,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6.947966e+06,347630.972013,12.206783,5.574553e+06,0.736360,0.757242,0.736139,0.716628,4.854622e+08,3.091988e+09,...,2.079088,10.985689,7.996819,22.217634,1001.541199,1506.456126,27.425647,21.330889,71.595088,1.830447
4,7.096137e+06,347411.232273,12.186539,5.609510e+06,0.736089,0.757859,0.736249,0.715498,4.867661e+08,3.189506e+09,...,2.183282,8.159014,8.651837,21.157249,1001.690342,1515.527356,27.608612,21.117524,70.061650,1.898255
5,7.239848e+06,349584.100654,12.182243,5.648192e+06,0.735929,0.758347,0.735436,0.715372,5.225848e+08,3.093872e+09,...,2.403445,7.913561,9.551098,18.013769,1001.817868,1528.998642,27.762053,20.876766,68.552790,1.943125
6,7.349320e+06,344289.736627,12.154911,5.667961e+06,0.735757,0.758460,0.735472,0.714737,5.676413e+08,2.779340e+09,...,2.748397,10.350442,10.328300,14.644773,1001.940536,1542.131372,27.986329,20.647922,66.896135,1.956684
7,7.439258e+06,340498.356490,12.130631,5.685185e+06,0.735070,0.758081,0.733678,0.714965,5.179392e+08,2.561731e+09,...,2.501529,12.454398,10.723767,19.007683,1001.988306,1541.272845,28.048825,20.665808,66.738045,1.910770
8,7.513614e+06,340512.038701,12.120014,5.705074e+06,0.735168,0.766432,0.738349,0.702779,4.524144e+08,2.516859e+09,...,2.073464,13.111908,10.764101,24.437747,1002.055065,1514.905960,27.959130,20.665948,67.018448,1.902067
9,7.560265e+06,340005.357917,12.115483,5.723617e+06,0.733637,0.765744,0.735380,0.701759,4.213689e+08,2.497710e+09,...,2.177386,9.479812,10.533542,26.318142,1002.155784,1495.611624,27.867978,20.547545,66.903661,1.943183


In [14]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      63.064000
1      86.408000
2     103.009000
3     100.066000
4     121.914000
5     120.417571
6     113.282057
7     105.481371
8      99.723000
9      83.012000
10    121.120000
11    151.571000
12    144.989000
13    132.217000
14           NaN
Name: Roraima - Consumo de Cimento (t), dtype: float64

In [15]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado,Roraima - IDH,Roraima - IDH Longevidade,Roraima - IDH Educacao,Roraima - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Roraima - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-1.744850,-0.315007,1.401150,-1.746144,0.761548,-1.560951,-0.502066,1.373638,1.204752,-0.990234,...,2.026352,0.984445,-1.243205,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-1.589344,0.383838,1.268386,-1.652750,0.786997,-1.123635,0.058230,1.051274,0.820657,-0.546688,...,0.085558,-0.051710,-1.216598,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-1.098918,1.403279,1.025732,-1.063622,0.770203,-0.776474,0.440255,0.792039,0.275392,0.158672,...,-1.114490,-0.115145,-1.248198,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.652507,1.029031,0.854618,-0.611278,0.712391,-0.517408,0.641649,0.594618,-0.132596,0.824346,...,-0.697344,0.633403,-1.258770,0.026787,-1.750903,0.264474,-1.694225,2.033190,1.942658,-2.299835
4,-0.239756,0.967268,0.468245,-0.216621,0.570378,-0.371368,0.691446,0.446582,-0.104860,1.148829,...,-0.337667,-0.646476,-0.700745,-0.229627,-1.027431,0.358296,-1.009920,1.276809,1.179290,-0.506518
5,0.160572,1.577997,0.386241,0.220096,0.487127,-0.255786,0.323198,0.430184,0.657046,0.830615,...,0.422337,-0.757614,0.065357,-0.989758,-0.408818,0.497626,-0.436039,0.423317,0.428157,0.680157
6,0.465520,0.089910,-0.135410,0.443290,0.396938,-0.229205,0.339660,0.346932,1.615453,-0.215960,...,1.613108,0.345772,0.727473,-1.804421,0.186229,0.633454,0.402771,-0.387939,-0.396551,1.038753
7,0.716056,-0.975734,-0.598807,0.637745,0.037917,-0.318769,-0.473299,0.376803,0.558230,-0.940033,...,0.760923,1.298413,1.064381,-0.749418,0.417955,0.624574,0.636511,-0.324530,-0.475251,-0.175540
8,0.923184,-0.971888,-0.801440,0.862290,0.089103,1.657894,1.643124,-1.219209,-0.835563,-1.089341,...,-0.716758,1.596124,1.098742,0.563636,0.741797,0.351869,0.301046,-0.324036,-0.335662,-0.405692
9,1.053138,-1.114301,-0.887917,1.071645,-0.710639,1.495023,0.298008,-1.352812,-1.495939,-1.153056,...,-0.358019,-0.048438,0.902324,1.018338,1.230371,0.152313,-0.039871,-0.743775,-0.392804,0.681701


In [16]:
reshaped_train_input = dfToInputRNN(train_input)

In [17]:
validation_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index - 1:-1]
validation_input = dfToInputRNN(validation_input)
validation_input

array([[[ 0.90685251, -1.00761554, -1.41116178,  0.9194875 ,
         -1.74185276,  0.80148276, -1.54447744, -1.37431419,
         -1.0453709 ,  1.21859064, -0.99976018,  0.79428371,
          1.15575749, -0.52559858,  1.30260559, -0.55253046,
         -0.25130359,  1.38787176,  1.47154451, -1.80649616,
          1.09296291,  0.94281138,  0.52836084,  0.61258103,
          0.87656257,  1.47801536,  1.71982086,  1.28221181,
          1.39086472,  1.58209844,  1.08872091,  0.55212962,
         -1.81398118,  1.44113674,  1.11191094,  1.30470835,
          0.86390516, -1.67365968, -1.70738939, -1.55813196,
         -0.48498301, -0.75714639,  0.40770893, -0.27394734,
          2.02323219,  1.2908951 ,  1.70465513,  1.81148084,
          1.39968653, -1.36191442,  1.28024593,  1.11054795,
          0.12662799,  1.40414112,  1.40468999,  1.45643959,
          1.58509336,  1.32283157,  1.30222047,  1.55138602,
          1.91801602, -0.77489784,  2.32098884, -1.1942541 ,
         -1.17081042,  0

In [18]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0      63.064000
1      86.408000
2     103.009000
3     100.066000
4     121.914000
5     120.417571
6     113.282057
7     105.481371
8      99.723000
9      83.012000
10    121.120000
11    151.571000
Name: Roraima - Consumo de Cimento (t), dtype: float64

In [19]:
# Alvo para validação do treinamento
validation_target = target_data.iloc[split_index - 1:-3]
validation_target

11    151.571
Name: Roraima - Consumo de Cimento (t), dtype: float64

In [20]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado,Roraima - IDH,Roraima - IDH Longevidade,Roraima - IDH Educacao,Roraima - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Roraima - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,0.646085,-0.836784,-1.404774,0.642293,-2.414957,0.237678,-2.61321,-1.325075,1.108405,2.635823,...,-1.482779,-2.260447,0.345514,-1.395304,-1.927921,-2.672061,2.471949,-1.925114,-2.328206,2.648601


In [21]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[ 0.64608546, -0.83678428, -1.40477375,  0.64229273,
         -2.41495677,  0.23767786, -2.61321023, -1.32507497,
          1.10840491,  2.63582309, -0.18995911,  0.72640489,
          2.66833308, -0.032632  ,  0.98802435,  0.24980494,
         -0.08955808,  1.85434394,  2.02706568, -2.21150703,
          1.42938591,  0.85132668,  0.14092386,  0.30902324,
          1.00267904,  2.27970137,  1.11272593,  0.61007902,
          0.89064538,  1.42231438,  1.02650562,  1.47127593,
         -1.53201473,  1.73503858,  1.66610613,  1.0456894 ,
          0.55289259, -1.81216593, -2.10342296, -2.55377478,
          0.41940628, -1.75756445,  0.241023  , -0.98659345,
          1.52184916,  1.01398607,  1.49290967,  1.40764092,
          1.87930411, -2.51892072,  2.83356214,  1.58620924,
         -0.02524128,  1.54231542,  1.5176119 ,  1.7355269 ,
          1.87941982,  1.54198129,  1.51276712,  1.83795594,
          2.62210693,  1.04993174,  0.96476493, -1.48277861,
         -2.26044738,  0

In [22]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    144.989
Name: Roraima - Consumo de Cimento (t), dtype: float64

In [23]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        tf.keras.utils.set_random_seed(seed)

    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.01
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(148, activation='tanh')),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        batch_size=train_input.shape[0],
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [24]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['loss'][500:])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [ ]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[2310321130, 3383184465, 2822723016, 96570311, 2974215513, 3326358546, 2074306289, 3855014413, 3609490585, 2513304765, 2459915000, 3126891873, 1307834335, 2973367552, 423382799, 3220668536, 3005615237, 2549935896, 2104640241, 3248983719, 2940204721, 1853560446, 1247259122, 3581824678, 1285629542, 1483926669, 3856509655, 2556817105, 2396108528, 866957385, 3757104407, 1408574253, 2010577604, 486607418, 3577564813, 1079616529, 3280311872, 3023776815, 1260899570, 1197578459, 948182294, 139695631, 1410694504, 113020200, 1402613605, 3155266624, 3038972174, 3543837584, 3854815370, 323744022, 418258175, 811281792, 758364072, 3844985449, 1701610091, 1869496400, 3144445484, 2322775838, 3490598541, 2748010764, 4058602087, 4162887578, 1998825830, 1659742670, 1214014204, 2352444849, 4063995980, 2302478914, 3495821292, 842526666, 2076836517, 3814007360, 3879280316, 225423986, 3473368238, 1114967591, 589183687, 1222966817, 1712010766, 3757928557, 3256754132, 1788398754, 4059410650, 3627586288, 155486

2023-09-10 12:10:05.331694: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-10 12:10:05.560217: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-10 12:10:05.560387: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

loss: nan


Step: 1 ___________________________________________
loss: nan


Step: 2 ___________________________________________
loss: nan


Step: 3 ___________________________________________
loss: nan


Step: 4 ___________________________________________
loss: nan


Step: 5 ___________________________________________
loss: nan


Step: 6 ___________________________________________
loss: nan


Step: 7 ___________________________________________
loss: nan


Step: 8 ___________________________________________
loss: nan


Step: 9 ___________________________________________
loss: nan


Step: 10 ___________________________________________
loss: nan


Step: 11 ___________________________________________
loss: nan


Step: 12 ___________________________________________
loss: nan


Step: 13 ___________________________________________
loss: nan


Step: 14 ___________________________________________
loss: nan


Step: 15 ___________________________________________
loss: nan


Step: 16 _____________

In [ ]:
trained_model, history = bidirectional_lstm_model(reshaped_train_input, 
                                                  train_target, want_verbose=1, seed=winner_seed)

In [ ]:
prediction = trained_model.predict(reshaped_test_input)[0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

In [ ]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

In [ ]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = bidirectional_lstm_model(dfToInputRNN(train_input), 
                                                     train_target, want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [ ]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

In [ ]:
display(targets)
display(predictions)

In [ ]:
mae = mean_absolute_error(predictions, targets)
mae

In [ ]:
porcentage = mae/np.mean(targets)
porcentage